# Topic Modeling with LDA

This tutorial shows how to perform LDA topic modeling using the `sklearn`. It uses different functions to explore the topics and the documents. It is using a small dataset from the NYT articles to keep the interpretation manageable. 

There are **a few scattered activities for you** in the notebook.


**Table of Content**

1. [Load the data](#sec1)  
2. [Convert to document-term matrix](#sec2)  
3. [Fit the LDA model and explore it](#sec3)
4. [Finding the most optimal number of topics with GridSearch](#sec4)

<a id="sec1"></a>
## 1. Load the data

I used the NYT API to get all articles from March 2024. Then, I combined together the fields "snippet" and "lead_paragraph" to create a longer document for each article. Then, I chose the articles for the section_name: food, realestate, and science. I saved the documents only into a json file. 

Below there is a function that will read a JSON file and turn it into a dataframe.

In [1]:
import json
import pandas as pd
import numpy as np

def jsonToDF(name):
    """Read a list of sentences from the JSON file, store them in a dataframe"""
    
    with open(f"{name}.json") as fin:
        textList = json.load(fin)

    # create a name for each document, based on its category
    indexNames = [f"{name}_{i+1}" for i in range(len(textList))]

    # create the dataframe, it will have one column and one index
    df = pd.DataFrame(data=textList, index=indexNames)
    df.columns = ['document']
    return df

First, let's read the content of all three files:

In [2]:
food = jsonToDF("food")
realestate = jsonToDF("realestate")
science = jsonToDF("science")

Check one dataframe:

In [3]:
food.head()

,document
food_1,Commit this method to memory for caramelized a...
food_2,"Kenji López-Alt’s buttery, unabashedly garlick..."
food_3,Just add rice or potatoes (and maybe a chilled...
food_4,"It’s a showstopping kaleidoscope of bulgogi, s..."
food_5,In the third installment of her YouTube series...


Check the shape of each dataframe:

In [4]:
print("food:", food.shape)
print("realestate:", realestate.shape)
print("science:", science.shape)

food: (81, 1)
realestate: (82, 1)
science: (71, 1)


Let's concatenate all of them in a single dataframe for the moment:

In [5]:
allDocs = pd.concat([food, realestate, science])
allDocs.shape

(234, 1)

In [6]:
allDocs.head()

,document
food_1,Commit this method to memory for caramelized a...
food_2,"Kenji López-Alt’s buttery, unabashedly garlick..."
food_3,Just add rice or potatoes (and maybe a chilled...
food_4,"It’s a showstopping kaleidoscope of bulgogi, s..."
food_5,In the third installment of her YouTube series...


Make the column wide enough to show all text:

In [7]:
pd.set_option("display.max_colwidth",1000)

Look at the results:

In [8]:
allDocs.head()

,document
food_1,"Commit this method to memory for caramelized and crisp yet tender vegetables all year long. The kindest thing you can do for yourself when you’re stiff from being in the cold is to find some warmth: Because as the chill in your bones starts to fade, so does your stiffness. The same thing happens to hard winter vegetables when they’re enveloped in the heat of the oven — they soften and sweeten as they roast until they’re golden outside and tender in the middle."
food_2,"Kenji López-Alt’s buttery, unabashedly garlicky noodles are as easy to make as they are to devour. Good morning. The vernal equinox is in less than three weeks, but you wouldn’t know it from the frosted mud in the woods and the storm-wounded lawns where I stay. It’s bare ugly everywhere save in the bays, where water clear as gin flows over rocks in a spectrum of pink. At the market: cabbage and potatoes, a box of turnips, industrial berries that might have been grown in space. The new season’s coming, sure as tulips, but right now it’s hard to imagine."
food_3,"Just add rice or potatoes (and maybe a chilled white wine). Citrus and salmon is a winning combination, previously proven in New York Times Cooking’s recipes for broiled salmon with mustard and lemon, roasted salmon with ginger-lime butter and citrusy roasted salmon and potatoes. Our newest addition to this esteemed company is Farideh Sadeghin’s recipe for orange-glazed baked salmon. It’s a no-fuss fish dinner with a clever, timesaving twist: Farideh builds a side salad into the recipe by tossing salad greens with some of the reserved honeyed orange juice that is used to flavor the salmon. If you’re looking at the above image and thinking, “I bet blood oranges would be especially beautiful and excellent in this recipe,” know that Coco, a reader, already tried that and can confirm that the results were “absolutely delicious.”"
food_4,"It’s a showstopping kaleidoscope of bulgogi, shiitakes, bean sprouts, spinach, carrots and cucumbers, all drizzled with a spicy gochujang sauce. Good morning. On Sunday, I like a project in the kitchen more than on any other day. It’s a chance to work at the stove without the need to get something on the table in 45 minutes, a time to stretch my skill set. Mostly, it’s an opportunity to explore recipes rather than simply following them. On Sundays I don’t want to fly by wire. I want to fly."
food_5,"In the third installment of her YouTube series, the cookbook author and chef Sohla El-Waylly will teach you how to master the basics of the bird. For beginners and experienced cooks alike, preparing chicken can come with a lot of questions (and nerves!). Am I going to get salmonella? How do I butcher a whole bird? Am I doomed to an eternity of dry breast meat? In the third installment of her YouTube series, Cooking 101, the chef and cookbook author Sohla El-Waylly will help you master the basics of the bird, then set you up with a handful of recipes that highlight white and dark meat."


In [9]:
allDocs.tail()

,document
science_67,"Dr. Goodall, who is best known for her work with chimpanzees, recently celebrated her forthcoming 90th birthday with as many dogs and explained why she isn’t slowing down. Jane Goodall is turning 90 on April 3 and the primatologist-turned-activist seems busier than ever. This year, she’ll be on the road for 320 days. She’ll be raising money for her nonprofit organizations, the Jane Goodall Institute and Roots & Shoots, and encouraging people to take environmental action."
science_68,How do champion skaters accomplish their extraordinary jumps and spins? Brain science is uncovering clues.
science_69,"The Delta IV Heavy, a rocket that briefly bursts into flame just before it lifts off, is set to launch for the last time soon. The ignition of the Delta IV Heavy rocket is perhaps the most visually striking liftoff you’ll ever see — the rocket seemingly burns itself up on the launchpad before it heads to space. Now, the very last Delta IV Heavy ever is on the launchpad."
science_70,"A device called LightSound is being distributed to help the blind and visually impaired experience this year’s event. On Aug. 21, 2017, Kiki Smith’s teenage sons giddily prepared to watch the partial solar eclipse in Rochester, N.Y. As Ms. Smith listened to their chatter, she felt excluded."
science_71,"The rendezvous between the sun and the moon in 2017 captivated a small region in the Midwest. Lucky for Americans at the eclipse crossroads, they get to see it again. It is rare for a total solar eclipse to hit the same place twice — once every 366 years on average. In 2019, this happened in the Pacific Ocean, far west of the coast of Chile. By a stroke of luck, the next one will span a region of about 10,000 square miles that includes parts of southern Illinois, southeastern Missouri and western Kentucky."


<a id="sec2"></a>
## 2. Convert to document-term matrix

We will apply the CountVectorizer to convert our corpus into a document-term matrix. Empirical evidence has shown that simply counting words is more meaningful for performing LDA on documents. (It is possible to use the Tf-idf vectorizer too.)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

This process has always two steps: 

1. initialize the vectorizer constructor
2. apply `fit_transform` to perform the transformation.

In [11]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(allDocs['document'])
dtm

<234x4504 sparse matrix of type '<class 'numpy.int64'>'
	with 8227 stored elements in Compressed Sparse Row format>

### Exploring the features   
Let's look at the features of the "model", that is, the columns of our document-term matrix:

In [14]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['abel', 'able', 'aboard', ..., 'zootampa', 'zumper', 'zuni'],
      dtype=object)

It's an array, let's look at its dimensions:

In [15]:
feature_names.shape

(4504,)

Let's look at a larger chunk of values:

In [16]:
feature_names[300:350]

array(['barge', 'bargoer', 'bargoers', 'barked', 'barley', 'barn', 'barr',
       'barriers', 'bars', 'bartender', 'bartered', 'bartlett', 'based',
       'basement', 'basic', 'basics', 'basil', 'basilica', 'basin',
       'basmati', 'bass', 'bassin', 'batch', 'bath', 'bathroom',
       'bathrooms', 'battle', 'bay', 'bays', 'beach', 'beaches', 'beads',
       'bean', 'beans', 'beat', 'beau', 'beautiful', 'beauty', 'beckoned',
       'bedford', 'bedrock', 'bedroom', 'bedrooms', 'beef', 'beefbars',
       'beekman', 'beets', 'began', 'beginners', 'beginning'],
      dtype=object)

It's clear that these are all cleaned words, three or more characters long, which have not been stemmed. That is, we have both "bean" and "beans" as two separate features.

### Understanding the document-term matrix

Let's look at a single row of the matrix, the first row, which corresponds to the first document from the NYT articles:

In [17]:
doc1 = dtm[0]
doc1

<1x4504 sparse matrix of type '<class 'numpy.int64'>'
	with 32 stored elements in Compressed Sparse Row format>

It says that it has 4504 colums, but there are only 32 stored elements (terms that are non-zero).

We can use some Python code to find the words and their counts for this document:

In [18]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

for word, score in zip(words, dtm_scores):
    print(f"{word}: {score}")

bones: 1
caramelized: 1
chill: 1
cold: 1
commit: 1
crisp: 1
does: 1
enveloped: 1
fade: 1
golden: 1
happens: 1
hard: 1
heat: 1
kindest: 1
long: 1
memory: 1
method: 1
middle: 1
outside: 1
oven: 1
roast: 1
soften: 1
starts: 1
stiff: 1
stiffness: 1
sweeten: 1
tender: 2
thing: 2
vegetables: 2
warmth: 1
winter: 1
year: 1


We can look at non_zero_indices to check what that variable stores:

In [19]:
non_zero_indices

array([ 437,  599,  713,  792,  830,  972, 1168, 1331, 1436, 1717, 1807,
       1810, 1840, 2162, 2334, 2503, 2513, 2525, 2792, 2794, 3392, 3728,
       3832, 3863, 3864, 3967, 4027, 4051, 4282, 4357, 4424, 4480])

These values correspond to the column indices of each of the terms (words) in the matrix. A word like "year" has a high index, since is toward the end of the matrix, where terms are ordered alphabetically. 

Now that we know the indices of these words, we can use them to find how often each words occurrs in the entire matrix.

We will check the word "caramelized", which has the index 599.

In [20]:
dtm.getcol(599).toarray().T # get the column, turn it into an array format, then transpose it to be a row

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

It's obvious that the word doesn't show up often, I see only 3 values of 1. Let's check for the word "vegetables", index = 4282

In [21]:
dtm.getcol(4282).toarray().T

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Even this word doesn't show in more than 3 documents in total. Meanwhile, let's see a word like "year", index = 4480:

In [22]:
dtm.getcol(4480).toarray().T

array([[1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

This seems to occur a bit more often, we can find in how many documents:

In [23]:
np.count_nonzero(dtm.getcol(4480).toarray().T)

18

### **Task for you:** find the top 5 words from this document (meaning, they show in most articles).

Use the variable names that have been seen so far.

In [50]:
count_df = pd.DataFrame(columns=['word', 'count'])
for count in range(dtm.shape[1]):
    word = feature_names[count]
    temp_count = np.count_nonzero(dtm.getcol(count).toarray().T)
    count_df.loc[len(count_df.index)] = [word, temp_count]
count_df.head()

,word,count
0,abel,1
1,able,2
2,aboard,1
3,aboveground,1
4,absolutely,2


In [58]:
count_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)[:5]

,word,count
0,new,69
1,home,47
2,years,30
3,just,27
4,like,27


In [63]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

score_list = [(word, score) for i, (word, score) in enumerate(zip(words, dtm_scores))]
score_df = pd.DataFrame(score_list, columns =['word', 'count'])

In [64]:
score_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)[:5]

,word,count
0,vegetables,2
1,thing,2
2,tender,2
3,bones,1
4,commit,1


**Reflection questions** 

1. From these top words would you be able to infer that this document is about cooking/food? 

2. If these words were part of a **topic**, what would you name that topic?

Tayae answers:
(Based on score_df/the cell right above this one)
1. Yes, these words are mostly be about food, although because their counts are so low, 
2. Food!

### Going back to the dataframe

We can create a function that takes the representation of each document as a row of numbers in the matrix and converts it back to a list of words.

In [65]:
def matrix2Doc(dtMatrix, features, index):
    """Turns each row of the document-term matrix into a list of terms"""
    row = dtMatrix.getrow(index).toarray()
    non_zero_indices = row.nonzero()[1]
    words = [features[idx] for idx in non_zero_indices]
    return words

In [66]:
allDocsAsTerms = [matrix2Doc(dtm, feature_names, i) for i in range(dtm.shape[0])]

Check that we have all of them:

In [67]:
len(allDocsAsTerms)

234

Add a column to the dataframe:

In [68]:
allDocs['terms'] = allDocsAsTerms
allDocs.head()

,document,terms
food_1,"Commit this method to memory for caramelized and crisp yet tender vegetables all year long. The kindest thing you can do for yourself when you’re stiff from being in the cold is to find some warmth: Because as the chill in your bones starts to fade, so does your stiffness. The same thing happens to hard winter vegetables when they’re enveloped in the heat of the oven — they soften and sweeten as they roast until they’re golden outside and tender in the middle.","[bones, caramelized, chill, cold, commit, crisp, does, enveloped, fade, golden, happens, hard, heat, kindest, long, memory, method, middle, outside, oven, roast, soften, starts, stiff, stiffness, sweeten, tender, thing, vegetables, warmth, winter, year]"
food_2,"Kenji López-Alt’s buttery, unabashedly garlicky noodles are as easy to make as they are to devour. Good morning. The vernal equinox is in less than three weeks, but you wouldn’t know it from the frosted mud in the woods and the storm-wounded lawns where I stay. It’s bare ugly everywhere save in the bays, where water clear as gin flows over rocks in a spectrum of pink. At the market: cabbage and potatoes, a box of turnips, industrial berries that might have been grown in space. The new season’s coming, sure as tulips, but right now it’s hard to imagine.","[alt, bare, bays, berries, box, buttery, cabbage, clear, coming, devour, easy, equinox, flows, frosted, garlicky, gin, good, grown, hard, imagine, industrial, kenji, know, lawns, lopez, make, market, morning, mud, new, noodles, pink, potatoes, right, rocks, save, season, space, spectrum, stay, storm, sure, tulips, turnips, ugly, unabashedly, vernal, water, weeks, woods, wouldn, wounded]"
food_3,"Just add rice or potatoes (and maybe a chilled white wine). Citrus and salmon is a winning combination, previously proven in New York Times Cooking’s recipes for broiled salmon with mustard and lemon, roasted salmon with ginger-lime butter and citrusy roasted salmon and potatoes. Our newest addition to this esteemed company is Farideh Sadeghin’s recipe for orange-glazed baked salmon. It’s a no-fuss fish dinner with a clever, timesaving twist: Farideh builds a side salad into the recipe by tossing salad greens with some of the reserved honeyed orange juice that is used to flavor the salmon. If you’re looking at the above image and thinking, “I bet blood oranges would be especially beautiful and excellent in this recipe,” know that Coco, a reader, already tried that and can confirm that the results were “absolutely delicious.”","[absolutely, add, addition, baked, beautiful, bet, blood, broiled, builds, butter, chilled, citrus, citrusy, clever, coco, combination, company, confirm, cooking, delicious, dinner, especially, esteemed, excellent, farideh, fish, flavor, fuss, ginger, glazed, greens, honeyed, image, juice, just, know, lemon, lime, looking, maybe, mustard, new, newest, orange, oranges, potatoes, previously, proven, reader, recipe, recipes, reserved, results, rice, roasted, sadeghin, salad, salmon, thinking, times, timesaving, tossing, tried, twist, used, white, wine, winning, york]"
food_4,"It’s a showstopping kaleidoscope of bulgogi, shiitakes, bean sprouts, spinach, carrots and cucumbers, all drizzled with a spicy gochujang sauce. Good morning. On Sunday, I like a project in the kitchen more than on any other day. It’s a chance to work at the stove without the need to get something on the table in 45 minutes, a time to stretch my skill set. Mostly, it’s an opportunity to explore recipes rather than simply following them. On Sundays I don’t want to fly by wire. I want to fly.","[bean, bulgogi, carrots, chance, cucumbers, day, don, drizzled, explore, fly, following, gochujang, good, kaleidoscope, kitchen, like, minutes, morning, need, opportunity, project, recipes, sauce, set, shiitakes, showstopping, simply, skill, spicy, spinach, sprouts, stove, stretch, sunday, sundays, table, time, want, wire, work]"
food_5,"In the third installment of h

<a id="sec3"></a>
## 3. Fit the LDA model

Now that the data is ready and we understand well how it is represented (and how sparse it is), let us fit the LDA model:

In [69]:
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Initialize the model

lda = LatentDirichletAllocation(n_components=15, # we are picking the number of topics arbitrarely at the moment
                                random_state=0)

# Step 2: Fit the model
lda.fit(dtm)

LatentDirichletAllocation(n_components=15, random_state=0)

The representation of topics can be accessed this way:

In [70]:
lda.components_

array([[0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 1.06666664,
        0.06666667],
       [0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667],
       [0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 0.06666667,
        2.06666667],
       ...,
       [0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667],
       [0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667],
       [0.06666667, 0.06666667, 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667]])

What are the dimensions?

In [71]:
lda.components_.shape

(15, 4504)

So, this is a 15 by 4504 matrix, where each row is one of our topics and each column is a word (term). The values that we see are **not** probabilities, they are the **parameters** fitted by the LDA model for the topic-term distribution. We can see that they are not probabilities, since at least some of them seem to have a value > 1. 

These values are so-called "pseudo-counts" that reflect how many times, probabilistically speaking, each word was assigned to each topic across the entire corpus, adjusted by the model's learning process. The values are proportional to the probability of a term given a topic, but they need to be normalized to sum to one across each row to represent actual probabilities.

Now that we have such a **topic-term distribution**, we can find the top words associated with each topic.

In [72]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 15)

Topic 0:
moon home world years major built spacecraft head year ago way place skaters champion include
Topic 1:
new city dining home restaurants year house space said restaurant built program years make victorian
Topic 2:
eclipse chicken restaurants new time built monday family home square near place moon style hard
Topic 3:
new rent apartment make building bedroom halloumi home biscuits work century time minute best high
Topic 4:
eggs home just easy sweet people early outside needs park probably day better founded cooking
Topic 5:
home sellers realtors estate real chicken group pay association new national said commissions lawsuits brought
Topic 6:
home bread kenji buy south american ago restaurant drive day species dishes years questions art
Topic 7:
bedroom room properties house space week new com bath kitchen half dining floor basement living
Topic 8:
family home salmon flour near recipe make leeks new work died like just small lower
Topic 9:
new home national united bedroom work s

**To note:** Looking at these words, it is hard to decide what topic each of them represents since words about food, realestate, and science are mixed together in each topic. Topic 11 seems relatively homogenous, it's clear that it is talking about food. 

Knowing how sparse our document-term matrix was (only 234 documents, but 4504 terms) it is to be expected that there isn't enough data to learn a better model that captures better topics (and the words associated with them).

### The document-topic matrix and dominant topics

In the prior step, by fitting the LDA model, we found the topics that are present in our corpus. Now, we will use these topics to generate the documents. For that, we will use the method `transform`. This method will transform our document-term matrix into a new matrix, the document-topic matrix. This is where the **dimensionality reduction** is happening. We go from the large document-term matrix to a narrow document-topic matrix.

In [74]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

array([[1.85185343e-03, 1.85185310e-03, 1.85185415e-03, ...,
        1.85185194e-03, 1.85185449e-03, 1.85185232e-03],
       [1.25786180e-03, 9.82389927e-01, 1.25786235e-03, ...,
        1.25786176e-03, 1.25786520e-03, 1.25786206e-03],
       [8.13008233e-04, 8.13008358e-04, 8.13008488e-04, ...,
        8.13008322e-04, 8.13008332e-04, 8.13008816e-04],
       ...,
       [2.29885068e-03, 2.29885189e-03, 2.29885136e-03, ...,
        9.67816078e-01, 2.29885078e-03, 2.29885116e-03],
       [2.29885133e-03, 9.67816079e-01, 2.29885285e-03, ...,
        2.29885289e-03, 2.29885103e-03, 2.29885086e-03],
       [1.58730305e-03, 1.58730257e-03, 9.77777766e-01, ...,
        1.58730338e-03, 1.58730263e-03, 1.58730230e-03]])

Verify the shape:

In [75]:
doc_topic_dist.shape

(234, 15)

**Meaning of the matrix values:** The entries in this matrix represent the proportion of the document's content that is attributed to each topic. This means each row of the output matrix is a distribution over topics for the corresponding document and should sum to one. We can easily test that by getting the sum of a row:

**Better representing the document-topic matrix**

The document-topic matrix above is not very legible, we will create a dataframe that has a better representation. First, I'll modify the function `display_topics` to show a few terms for each topic:

In [76]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [77]:
# column names
topicnames = displayHeader(lda, feature_names, 5)

# index names
docnames = allDocs.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

,"Topic 0: moon, home, world, years, major","Topic 1: new, city, dining, home, restaurants","Topic 2: eclipse, chicken, restaurants, new, time","Topic 3: new, rent, apartment, make, building","Topic 4: eggs, home, just, easy, sweet","Topic 5: home, sellers, realtors, estate, real","Topic 6: home, bread, kenji, buy, south","Topic 7: bedroom, room, properties, house, space","Topic 8: family, home, salmon, flour, near","Topic 9: new, home, national, united, bedroom","Topic 10: like, new, spring, cooking, winter","Topic 11: sheet, pan, new, eggs, meal","Topic 12: new, birds, known, sea, low","Topic 13: bond, trees, years, mortgage, year","Topic 14: new, study, family, percent, estate",dominant_topic
food_1,0.002,0.002,0.002,0.002,0.974,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,4
food_2,0.001,0.982,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1
food_3,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.989,0.001,0.001,0.001,0.001,0.001,0.001,8
food_4,0.002,0.002,0.002,0.978,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,3
food_5,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.982,0.001,0.001,0.001,11


Let's look at some documents between food and realestate:

In [78]:
df_document_topic[76:86]

,"Topic 0: moon, home, world, years, major","Topic 1: new, city, dining, home, restaurants","Topic 2: eclipse, chicken, restaurants, new, time","Topic 3: new, rent, apartment, make, building","Topic 4: eggs, home, just, easy, sweet","Topic 5: home, sellers, realtors, estate, real","Topic 6: home, bread, kenji, buy, south","Topic 7: bedroom, room, properties, house, space","Topic 8: family, home, salmon, flour, near","Topic 9: new, home, national, united, bedroom","Topic 10: like, new, spring, cooking, winter","Topic 11: sheet, pan, new, eggs, meal","Topic 12: new, birds, known, sea, low","Topic 13: bond, trees, years, mortgage, year","Topic 14: new, study, family, percent, estate",dominant_topic
food_77,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.973,0.002,0.002,0.002,0.002,0.002,0.002,8
food_78,0.006,0.006,0.006,0.006,0.915,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,4
food_79,0.001,0.001,0.001,0.001,0.984,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,4
food_80,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.965,0.002,0.002,0.002,0.002,10
food_81,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.983,0.001,0.001,0.001,0.001,0.001,9
realestate_1,0.002,0.002,0.002,0.972,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,3
realestate_2,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.928,0.005,0.005,0.005,0.005,0.005,0.005,8
realestate_3,0.963,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0
realestate_4,0.963,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0
realestate_5,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.839,0.145,13


One interesting thing here is that articles food_78 and food_79 seem to share the dominant topic, just like realestate_1 and realestate_2. Interestingly, realestate_5 has two topics with value > 0.1, both of which seem to be primarely about real estate.

### Topic distribution across documents

Now that we have the document-topic matrix, we can see which topics show up most frequently:

In [79]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,4,23
1,3,23
2,5,23
3,7,22
4,8,19
5,10,15
6,13,15
7,6,13
8,9,13
9,1,13


### Challenge yourself: Add two more columns

Using your pandas skills add two new columns to this dataframe:

1. a column with the top 10 words of the corresponding topic. (see Topic Num for the topic number)
2. a column that lists the document names associated with the topic (document names are things like food_1, food_2, etc.)

By adding these two columns it will be a bit easier to understand what is going one with the model and whether it is capturing something about the corpus of documents. 

1. a column with the top 10 words of the corresponding topic. (see Topic Num for the topic number)

In [95]:
topic_top10words_df = pd.DataFrame(columns =['Topic Num', 'Topic - Top 10 Words'])
for topic in displayHeader(lda, feature_names, 10):
    topic_num = topic.split(':')[0].split()[1]
    topic_list = topic.split(':')[1]
    topic_top10words_df.loc[len(topic_top10words_df.index)] = [topic_num, topic_list]
topic_top10words_df

,Topic Num,Topic - Top 10 Words
0,0,"moon, home, world, years, major, built, spacecraft, head, year, ago"
1,1,"new, city, dining, home, restaurants, year, house, space, said, restaurant"
2,2,"eclipse, chicken, restaurants, new, time, built, monday, family, home, square"
3,3,"new, rent, apartment, make, building, bedroom, halloumi, home, biscuits, work"
4,4,"eggs, home, just, easy, sweet, people, early, outside, needs, park"
5,5,"home, sellers, realtors, estate, real, chicken, group, pay, association, new"
6,6,"home, bread, kenji, buy, south, american, ago, restaurant, drive, day"
7,7,"bedroom, room, properties, house, space, week, new, com, bath, kitchen"
8,8,"family, home, salmon, flour, near, recipe, make, leeks, new, work"
9,9,"new, home, national, united, bedroom, work, science, just, earth, year"


In [99]:
topic_top10words_df['Topic Num']=topic_top10words_df['Topic Num'].astype(int)
df_topic_distribution['Topic Num']=df_topic_distribution['Topic Num'].astype(int)

In [100]:
df_topic_distribution2 = pd.merge(df_topic_distribution, topic_top10words_df, on='Topic Num')
df_topic_distribution2

,Topic Num,Num Documents,Topic - Top 10 Words
0,4,23,"eggs, home, just, easy, sweet, people, early, outside, needs, park"
1,3,23,"new, rent, apartment, make, building, bedroom, halloumi, home, biscuits, work"
2,5,23,"home, sellers, realtors, estate, real, chicken, group, pay, association, new"
3,7,22,"bedroom, room, properties, house, space, week, new, com, bath, kitchen"
4,8,19,"family, home, salmon, flour, near, recipe, make, leeks, new, work"
5,10,15,"like, new, spring, cooking, winter, york, clubs, recipe, day, dinner"
6,13,15,"bond, trees, years, mortgage, year, company, lease, new, old, food"
7,6,13,"home, bread, kenji, buy, south, american, ago, restaurant, drive, day"
8,9,13,"new, home, national, united, bedroom, work, science, just, earth, year"
9,1,13,"new, city, dining, home, restaurants, year, house, space, said, restaurant"


2. a column that lists the document names associated with the topic (document names are things like food_1, food_2, etc.)

In [174]:
topic_docname_dict = {k:[] for k in list(range(15))}
for row in range(df_document_topic['dominant_topic'].size):
    topic_num = df_document_topic['dominant_topic'].iloc[row]
    row_name = df_document_topic.iloc[row].name
    topic_docname_dict[topic_num].append(row_name)
topic_docname_df = df = pd.DataFrame(topic_docname_dict.items(), columns=['Topic Num', 'Document Names'])

In [175]:
topic_docname_df['Topic Num']=topic_docname_df['Topic Num'].astype(int)

In [176]:
df_topic_distribution3 = pd.merge(df_topic_distribution2, topic_docname_df, on='Topic Num')
df_topic_distribution3

,Topic Num,Num Documents,Topic - Top 10 Words,Document Names
0,4,23,"eggs, home, just, easy, sweet, people, early, outside, needs, park","[food_1, food_30, food_38, food_51, food_52, food_56, food_61, food_65, food_71, food_78, food_79, realestate_11, realestate_16, realestate_36, realestate_38, realestate_50, realestate_52, science_6, science_12, science_23, science_41, science_43, science_61]"
1,3,23,"new, rent, apartment, make, building, bedroom, halloumi, home, biscuits, work","[food_4, food_11, food_23, food_25, food_33, food_36, food_57, food_62, food_75, realestate_1, realestate_12, realestate_27, realestate_34, realestate_42, realestate_45, realestate_51, realestate_66, realestate_69, realestate_77, realestate_78, science_4, science_52, science_67]"
2,5,23,"home, sellers, realtors, estate, real, chicken, group, pay, association, new","[food_17, food_18, food_44, food_46, food_47, food_49, food_68, food_76, realestate_8, realestate_9, realestate_13, realestate_24, realestate_40, realestate_43, realestate_55, realestate_60, realestate_70, realestate_71, realestate_79, science_16, science_28, science_55, science_56]"
3,7,22,"bedroom, room, properties, house, space, week, new, com, bath, kitchen","[food_10, food_22, food_54, food_63, realestate_17, realestate_19, realestate_25, realestate_26, realestate_28, realestate_32, realestate_33, realestate_35, realestate_37, realestate_53, realestate_54, realestate_64, realestate_74, realestate_75, science_1, science_15, science_42, science_66]"
4,8,19,"family, home, salmon, flour, near, recipe, make, leeks, new, work","[food_3, food_20, food_39, food_50, food_53, food_66, food_77, realestate_2, realestate_18, realestate_21, realestate_44, realestate_56, realestate_65, science_7, science_11, science_18, science_25, science_46, science_68]"
5,10,15,"like, new, spring, cooking, winter, york, clubs, recipe, day, dinner","[food_6, food_9, food_28, food_48, food_58, food_59, food_69, food_80, realestate_80, science_14, science_29, science_39, science_49, science_50, science_59]"
6,13,15,"bond, trees, years, mortgage, year, company, lease, new, old, food","[food_21, food_24, food_26, food_37, food_40, food_43, food_70, realestate_5, realestate_48, realestate_82, science_21, science_22, science_31, science_32, science_37]"
7,6,13,"home, bread, kenji, buy, south, american, ago, restaurant, drive, day","[food_29, food_32, food_42, food_45, food_60, realestate_14, realestate_31, realestate_59, science_8, science_9, science_20, science_44, science_48]"
8,9,13,"new, home, national, united, bedroom, work, science, just, earth, year","[food_31, food_34, food_81, realestate_10, realestate_15, realestate_49, realestate_73, realestate_76, science_19, science_27, science_34, science_35, science_58]"
9,1,13,"new, city, dining, home, restaurants, year, house, space, said, restaurant","[food_2, food_15, food_19, food_27, food_41, realestate_6, realestate_46, realestate_57, science_24, science_45, science_54, science_63, science_70]"


### Interpretation Task

Pick a topic that doesn't have many documents assigned to it and then read all the articles (see dataframe at the start of the notebook) associated with this topic. Do you see any reason for why they were given the same dominant topic? Can you summarize in a single phrase what the meaning of that topic is? (Also make use of the top 15 words for that topic.)

Reading articles

In [212]:
index_num = df_topic_distribution3.index.get_loc(df_topic_distribution3[df_topic_distribution3['Topic Num'] == 11].index[0])
doc_list = df_topic_distribution3.loc[index_num, 'Document Names']

In [213]:
allDocs.loc['food_5']['document']

'In the third installment of her YouTube series, the cookbook author and chef Sohla El-Waylly will teach you how to master the basics of the bird. For beginners and experienced cooks alike, preparing chicken can come with a lot of questions (and nerves!). Am I going to get salmonella? How do I butcher a whole bird? Am I doomed to an eternity of dry breast meat? In the third installment of her YouTube series, Cooking 101, the chef and cookbook author Sohla El-Waylly will help you master the basics of the bird, then set you up with a handful of recipes that highlight white and dark meat.'

In [214]:
for doc in doc_list:
    full_doc = allDocs.loc[doc]['document']
    print(full_doc)

In the third installment of her YouTube series, the cookbook author and chef Sohla El-Waylly will teach you how to master the basics of the bird. For beginners and experienced cooks alike, preparing chicken can come with a lot of questions (and nerves!). Am I going to get salmonella? How do I butcher a whole bird? Am I doomed to an eternity of dry breast meat? In the third installment of her YouTube series, Cooking 101, the chef and cookbook author Sohla El-Waylly will help you master the basics of the bird, then set you up with a handful of recipes that highlight white and dark meat.
Sip teas inspired by Frank Lloyd Wright, learn to cook the cuisine of Punjab and more food news. Some cutting-edge appliances like the air fryer may have risen in popularity lately, but the simple sheet pan has become as essential as a cast-iron skillet. In the past, the sheet pan was often called a rimmed baking sheet and used in kitchens to make the cake known as chocolate roulade; it’s now all you need

Top 15 words

In [227]:
topic_top15words_dict = {x.split(':')[0]:x.split(':')[1].strip() for x in displayHeader(lda, feature_names, 15)}

In [228]:
topic_top15words_dict['Topic 11']

'sheet, pan, new, eggs, meal, dog, chicken, city, just, like, recipes, people, bird, skillet, dinner'

Based on these articles and this list of top 15 words, it's not entirely clear to me why these documents are grouped together. However, several of them seem to mention chicken/birds/eggs, and several of them have people's names and give narratives about these people's lives. So perhaps this is how the articles were grouped together.

<a id="sec4"></a>
## 4. Grid Search: Find number of topics

In the example so far, we arbitrarely chose the number of topics to be 15. However, that is not the right way to go about it. We whould use methods for selecting the optimal number of topics. This can be done through a mechanism known as GridSearch with cross-validation that builds multiple models and then compares them to see which one performs the best.

In [231]:
from sklearn.model_selection import GridSearchCV

# We are going to test multiple values for the number of topics
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'n_components': [5, 10, 15, 20, 25, 30, 35]})

Let us look at the results:

In [232]:
grid.cv_results_

{'mean_fit_time': array([0.22863894, 0.20625629, 0.22637253, 0.22886901, 0.22979016,
        0.26801291, 0.27164297]),
 'std_fit_time': array([0.00948951, 0.00463259, 0.00651776, 0.00978289, 0.00578743,
        0.01719679, 0.00848216]),
 'mean_score_time': array([0.01062899, 0.01132512, 0.01303353, 0.01265621, 0.01377544,
        0.0119853 , 0.012609  ]),
 'std_score_time': array([0.00096381, 0.00089207, 0.00109123, 0.00092054, 0.00048298,
        0.00071341, 0.00048343]),
 'param_n_components': masked_array(data=[5, 10, 15, 20, 25, 30, 35],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_components': 5},
  {'n_components': 10},
  {'n_components': 15},
  {'n_components': 20},
  {'n_components': 25},
  {'n_components': 30},
  {'n_components': 35}],
 'split0_test_score': array([-31566.74577891, -40071.12282821, -47366.98591621, -54458.79800951,
        -61328.53860792, -67744.32700419, -74630.788261

Since this representation is a bit overwhelming, let's access a few features of the grid instance:

In [233]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'n_components': 5}
Best Log Likelihood Score:  -26137.634101962678
Model Perplexity:  4688.161709810889


The results are showing that the best LDA model should have 5 topics, the smallest number we tried. This raises the question of whether we should try other small numbers, which I'm doing below:

In [234]:
search_params = {'n_components': [1,2,3,4,5,6]}

lda = LatentDirichletAllocation()
grid = GridSearchCV(lda, param_grid=search_params)

grid.fit(dtm)

# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'n_components': 1}
Best Log Likelihood Score:  -18224.96141745928
Model Perplexity:  4147.085591679493


This result shows that actually the best number of topics for this corpus is 1.

**Meaning of Log Likelihood**. 

Log Likelihood is the logarithm of the probability of observing the given data under the model with specific parameters. Essentially, it measures how well the model explains the observed data. (It is a conditional probability.)

**Meaning of perplexity**

Perplexity is a common metric used to evaluate the quality of probabilistic models. It reflects how well the model describes or predicts the documents in the dataset.

A lower perplexity score suggests that the model is more certain about its predictions (i.e., the probability distributions it assigns to unseen documents are more accurate). This means that the topic distributions learned by the model are a good fit for the observed data.

**Words for best modesl with one topic**

Let's see what are the top words for the best model with one topic:

In [235]:
display_topics(best_lda_model, feature_names, 40)

Topic 0:
new home bedroom like years just house space family chicken week year city kitchen york time make estate eggs apartment real recipe work restaurants room cooking study people restaurant built known said ago american spring national near good day floor


As we can see it is a mix of food and realestae and New York. If we had documents with more distinct nature and more of them we might have seen something else. 

However, the point of this tutorial was to show the mechanics of building LDA models. 

Now it's time to take what you saw here and apply it to your projects.

Have fun exploring!